In [ ]:
import pandas as pd
import re
from tabulate import tabulate
import plotly.express as px

cl1 = "Filled by"
cl = "Select farmer highest education"
cl2 = "Are you Associated with this FPC"
df = pd.read_csv('/content/FPONext.csv')  #replace with your file

def standardize_name(name):
    if isinstance(name, str):  # Check if name is a string
        name = name.lower()
        name = re.sub(r'\b([a-z]) ', lambda x: x.group(1).upper() + " ", name)
    return name


df[cl1] = df[cl1].apply(standardize_name)

counts = df.groupby([cl1, cl2]).size().reset_index(name="Count")

pivot_table = counts.pivot_table(index=cl1, columns=cl2, values='Count', aggfunc='sum', fill_value=0)

pivot_table['Total'] = pivot_table.sum(axis=1)

table = pivot_table.reset_index().rename_axis(None, axis=1)

print(tabulate(table, headers='keys', tablefmt='double_outline'))


╔═════╦══════════════════════════════════════════════════╦═══════════╦═══════════╦═════════╗
║     ║ Filled by                                        ║   No (नहीं) ║   Yes (हाँ) ║   Total ║
╠═════╬══════════════════════════════════════════════════╬═══════════╬═══════════╬═════════╣
║   0 ║ aadesh patil 9689584273                          ║        37 ║         0 ║      37 ║
║   1 ║ aasef kadu sayyed 9860688690 9860688690          ║         1 ║         0 ║       1 ║
║   2 ║ abhishek kumar 9534857527                        ║         1 ║         0 ║       1 ║
║   3 ║ abodh kumar singh 8873553307                     ║         3 ║         3 ║       6 ║
║   4 ║ aditya kumar suman 8207309845                    ║         1 ║         0 ║       1 ║
║   5 ║ ahatesham ahmad qureshi 9424724640               ║         1 ║         0 ║       1 ║
║   6 ║ ajay dabhade 7498886994                          ║         1 ║         0 ║       1 ║
║   7 ║ akash kumar 9835758057                           ║        

In [ ]:
# Convert counts to percentages
pivot_table_percentage = pivot_table.div(pivot_table['Total'], axis=0) * 100
# Drop 'Total' column
pivot_table_percentage = pivot_table_percentage.drop(columns=['Total'])
# Add a column for total percentage
pivot_table_percentage['Total Percentage'] = pivot_table_percentage.sum(axis=1)
table = pivot_table_percentage.reset_index().rename_axis(None, axis=1)

print(tabulate(table, headers='keys', tablefmt='double_outline'))

╔═════╦══════════════════════════════════════════════════╦═══════════╦════════════╦════════════════════╗
║     ║ Filled by                                        ║   No (नहीं) ║    Yes (हाँ) ║   Total Percentage ║
╠═════╬══════════════════════════════════════════════════╬═══════════╬════════════╬════════════════════╣
║   0 ║ aadesh patil 9689584273                          ║  100      ║   0        ║                100 ║
║   1 ║ aasef kadu sayyed 9860688690 9860688690          ║  100      ║   0        ║                100 ║
║   2 ║ abhishek kumar 9534857527                        ║  100      ║   0        ║                100 ║
║   3 ║ abodh kumar singh 8873553307                     ║   50      ║  50        ║                100 ║
║   4 ║ aditya kumar suman 8207309845                    ║  100      ║   0        ║                100 ║
║   5 ║ ahatesham ahmad qureshi 9424724640               ║  100      ║   0        ║                100 ║
║   6 ║ ajay dabhade 7498886994                    

In [ ]:
fig = px.bar(counts, x=cl1, y='Count', color=cl2,
             title='Count of Select farmer highest education by Filled by',
             labels={cl1: cl1, 'Count': 'Count', cl2: 'Education Level'},
             width=1200, height=600)
fig.show()

In [ ]:
filled_by_counts = counts.groupby(cl1).size().reset_index(name="Filled by Count")
merged_counts = pd.merge(counts, filled_by_counts, on=cl1)
fig = px.sunburst(merged_counts, path=[cl1, cl2], values='Count',
                  title='Count of selected options of highest education by count of Filled by')
fig.show()

In [ ]:
education_counts = df[cl2].value_counts().reset_index()
education_counts.columns = ['Education', 'Count']
fig = px.pie(education_counts, values='Count', names='Education', title='Distribution of Select farmer highest education')
fig.update_traces(textinfo='percent+label')
fig.show()

In [ ]:
outliers = counts[counts['Count'] > counts['Count'].mean() + 2 * counts['Count'].std()]
# Find outliers where the count of unique occurrences is significantly higher than the average, we identify outliers where the count of occurrences is more than two standard deviations above the mean.
print(tabulate(outliers, headers='keys', tablefmt='double_outline'))

╔═════╦════════════════════════════════╦════════════════════════════════════╦═════════╗
║     ║ Filled by                      ║ Are you Associated with this FPC   ║   Count ║
╠═════╬════════════════════════════════╬════════════════════════════════════╬═════════╣
║  15 ║ amrita pritam 8581900586       ║ No (नहीं)                            ║      84 ║
║  47 ║ ganesh pawar 8698837112        ║ No (नहीं)                            ║      61 ║
║  61 ║ jay nagvanshi 8720030721       ║ No (नहीं)                            ║     143 ║
║  80 ║ mamta shende 9993687537        ║ No (नहीं)                            ║     110 ║
║  86 ║ murari kumar 6204562195        ║ No (नहीं)                            ║      57 ║
║  92 ║ nishant kumar rajak 7209472746 ║ No (नहीं)                            ║      84 ║
║ 115 ║ ram ahkey official 9109582643  ║ No (नहीं)                            ║      81 ║
║ 126 ║ rinku kumari 8210316450        ║ No (नहीं)                            ║     107 ║
╚═════╩═════════

In [ ]:
fig = px.box(counts, x=cl2, y="Count", color=cl1, title="Box Plot", points="all")
fig.update_xaxes(title_text=cl2)
fig.update_yaxes(title_text="Count")
fig.update_traces(marker=dict(size=8))

fig.add_hline(y=10, line_dash="solid", line_color="rgb(255, 0, 0)", line_width=3, annotation_text="Outlier Threshold", annotation_position="top left", annotation_font_size=12)
fig.show()



In [ ]:


# Calculate total count for each category in cl1
total_counts = df.groupby(cl1).size()

# Calculate percentages
counts['Percentage'] = counts.apply(lambda x: (x['Count'] / total_counts[x[cl1]]) * 100, axis=1)



In [ ]:
# Calculate the interquartile range (IQR)
Q1 = counts['Percentage'].quantile(0.1)
Q3 = counts['Percentage'].quantile(0.5)
IQR = Q3 - Q1

# Identify outliers
outliers = counts[(counts['Percentage'] < (Q1)) | (counts['Percentage'] > (Q3))]

print(tabulate(outliers, headers='keys', tablefmt='double_outline'))


╔═════╦═══════════════════════════════════════════════╦════════════════════════════════════╦═════════╦══════════════╗
║     ║ Filled by                                     ║ Are you Associated with this FPC   ║   Count ║   Percentage ║
╠═════╬═══════════════════════════════════════════════╬════════════════════════════════════╬═════════╬══════════════╣
║   8 ║ akash kumar 9835758057                        ║ No (नहीं)                            ║       1 ║    25        ║
║  16 ║ amrita pritam 8581900586                      ║ Yes (हाँ)                            ║       4 ║     4.54545  ║
║  26 ║ ashish kawde 8889239594                       ║ Yes (हाँ)                            ║       1 ║    14.2857   ║
║  30 ║ banti K 9407055236                            ║ Yes (हाँ)                            ║      10 ║    19.6078   ║
║  34 ║ bhurelal silu 9174684759                      ║ Yes (हाँ)                            ║       4 ║    13.7931   ║
║  48 ║ ganesh pawar 8698837112               